# II- Demo. Optimal Interpolation

An example of simulated SSH data access is provided in the "example_data_access_meom.ipynb" notebook. Here, an example of a mapping technique based on a simple optimal interpolation is proposed. The notebook is structured as follow: 

    1) set optimal interpolation parameters,
    2) reading of pseudo-observations,
    3) perform optimal interpolation and,
    4) save the results (reconstructed SSH field)


Here, we assume a vector of observations, noted $y$ defined as:

$$y = H x + \epsilon $$

where $H$ is a linear observation operator between the reconstruction grid space and the observation space
, $x$ is the state to estimate and $\epsilon$ is an independent observation error.

The optimal interpolation consists in estimating an analysed state $x_{a}$ in combining the available observations to approximate the real state $x$:

$$x_{a} = K y $$
where $K$ is the weigth matrix defined as:

$$ K = BH^T(HBH^T + R)^{-1} $$

$B$ is the covariance matrix of $x$, and $R$ the covariance matrix of the error vector $\epsilon$ ($^T$ is the transpose operator)

In [ ]:
import xarray as xr
import numpy
import warnings
import logging
import sys
import os
warnings.filterwarnings('ignore')

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
sys.path.append('..')

In [ ]:
from src.mod_oi import *
from src.mod_inout import *
from src.mod_regrid import *
from src.mod_eval import *
from src.mod_plot import *

### 1) set optimal interpolation parameters

In [ ]:
# OI Grid
lon_min = 295.                                           # domain min longitude
lon_max = 305.                                           # domain max longitude
lat_min = 33.                                            # domain min latitude
lat_max = 43.                                            # domain max latitude
time_min = numpy.datetime64('2012-10-22')                # domain min time
time_max = numpy.datetime64('2012-12-02')                # domain max time
dx = 0.2                                                 # zonal grid spatial step (in degree)
dy = 0.2                                                 # meridional grid spatial step (in degree)
dt = numpy.timedelta64(1, 'D')                           # temporal grid step

simu_start_date = '2012-10-01T00:00:00'                  # Nature run initial date

glon = numpy.arange(lon_min, lon_max + dx, dx)           # output OI longitude grid
glat = numpy.arange(lat_min, lat_max + dy, dy)           # output OI latitude grid
gtime = numpy.arange(time_min, time_max + dt, dt)        # output OI time grid

# OI parameters
Lx = 1.                                                  # Zonal decorrelation scale (in degree)
Ly = 1.                                                  # Meridional decorrelation scale (in degree)
Lt = 7.                                                  # Temporal decorrelation scale (in days)
noise = 0.05                                             # Noise level (5%)

### 2) reading of pseudo-observations + define output folder

In [ ]:
inputs = ['../dc_obs/2020a_SSH_mapping_NATL60_jason1.nc'] 

In [ ]:
# Load obs dataset
ds = xr.open_dataset(inputs[0])
ds

In [ ]:
# Define outputs
output_directory = '../results/'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)  
output_oi = f'../results/ssh_reconstruction_{time_min}-{time_max}_jason1.nc'

### 3) perform optimal interpolation

In [ ]:
%%time
# set OI param & grid
ds_oi1_param = oi_param(Lx, Ly, Lt, noise)
ds_oi1_grid = oi_grid(glon, glat, gtime, simu_start_date)
# Read input obs + discard a bit...
coarsening = {'time': 5}
ds_oi1_obs = read_obs(inputs, ds_oi1_grid, ds_oi1_param, simu_start_date, coarsening)
# Run OI
for it in range(len(gtime)):
    oi_core(it, ds_oi1_grid, ds_oi1_param, ds_oi1_obs)

### 4) save the results (reconstructed SSH field)

In [ ]:
ds_oi1_grid.to_netcdf(output_oi)